In [ ]:
import heapq as hq
from tqdm import tqdm
with open("input_18.") as f:
    input = f.read().split("\n")
len(input)

In [2]:
# reversed because I want matrix coordinates
walls = [tuple(reversed([int(j) for j in i.split(",")])) for i in input]

In [3]:
def dijkstra(graphlen, walls):

    directions = {(1,0), (-1,0), (0,1), (0,-1)}
    nodes = []
    z = 0

    for i in range(graphlen):
        node = []
        for j in range(graphlen):
            if (i,j) in walls:
                cost = float("inf")
            else: 
                cost = 1
            node.append({
                "cost": cost,
                "position":(i,j), 
                "distance": float("inf"),
                "predecessor": None,
            })
        nodes.append(node)
    nodes[0][0]["distance"] = 0
    heap = [(0, 0, nodes[0][0])]
    seen = set()
    
    while heap:
        _, _, current_node = hq.heappop(heap)
        current_position = current_node["position"]
        if current_position == (graphlen-1, graphlen-1): break
        if current_position in seen: continue
        seen.add(current_position)
        
        for y,x in directions:
            position = current_position[0]+y, current_position[1]+x
            if position[0] in range(graphlen) and position[1] in range(graphlen) and position != current_node["predecessor"]:
                node = nodes[position[0]][position[1]]
                dist = current_node["distance"] + node["cost"]
                
                if dist < node["distance"]:
                    nodes[position[0]][position[1]]["distance"] = dist
                    nodes[position[0]][position[1]]["predecessor"] = current_position
                    hq.heappush(heap, (dist, (z:=z+1), nodes[position[0]][position[1]]))
    return nodes   

In [ ]:
# part 1:
graph = dijkstra(71, walls[:1024])
graph[-1][-1]["distance"]

In [19]:
def trace_path(nodes):
    path = []
    current_node = nodes[-1][-1]
    while current_node != nodes[0][0]:
        path.append(current_node["position"])
        i, j = current_node["predecessor"]
        current_node = graph[i][j]
    path.append((0, 0))
    return path

In [30]:
def visualize_grid(nodes):
    graphlen = len(nodes)
    grid = []
    path = trace_path(nodes)
    for i in range(graphlen):
        line = []
        for j in range(graphlen):
            if (i,j) in path:
                line.append("O")
            else:
                line.append(".")
        grid.append(line)
    for line in grid:
        print("".join(line))

In [ ]:
visualize_grid(graph)

In [ ]:
# part 2:
path = trace_path(graph)

for i, wall in tqdm(enumerate(walls[1024:])):
    if wall in path: # blocked --> recalculate
        graph = dijkstra(71, walls[:1024+i+1])
        if graph[-1][-1]["distance"] == float("inf"):
            print(f"{wall[1]},{wall[0]}")
            break
        path = trace_path(graph)
